# Attentional Decoding

### Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import mne

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler

from mne.decoding import (CSP, SlidingEstimator, cross_val_multiscore, LinearModel,
                         GeneralizingEstimator, Scaler, get_coef, Vectorizer)
from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.viz.topomap import _prepare_topo_plot, plot_topomap
from mne import io, EvokedArray

from scipy.signal import welch



## Read EEG Data

In [ ]:
epochs = mne.read_epochs_eeglab(input_fname='../Results/Run4/fourSpeakerSimultaneous.set')
print(epochs)
print(epochs.event_id)
labels = (epochs.events[:, -1] -1)
print(labels)


## Train and Optimize a Classifier with Tuned Hyperparameters

In [ ]:
# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = epochs.get_data()
cv = ShuffleSplit(20, test_size=0.25, random_state=42)
cv_split = cv.split(epochs_data)



#params = {'C': [0.1, 0.5, 1, 2, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 2, 10]}
params = {'SVC__C': [0.1, 1, 2], 'SVC__gamma': [0.001, 0.01, 0.1, 1, 2, 3], 'CSP__n_components': [3,6,12]}
SVC = svm.SVC(kernel='rbf')
csp = CSP(log=True, norm_trace=False)
pipe = Pipeline([('CSP', csp), ('SVC', SVC)])
search = GridSearchCV(pipe, params, cv=cv, n_jobs=-1, iid=False, return_train_score=False)
search.fit(epochs_data, labels)
print('Classification Accuracy: ', round(search.best_score_,3), ' ---- Best Parameters:', 
      ' C: ', search.best_params_.get('SVC__C'), ' Gamma: ', search.best_params_.get('SVC__gamma'),
      ' # CSP_components: ', search.best_params_.get('CSP__n_components'))
csp = CSP(n_components=search.best_params_.get('CSP__n_components'), log=True, norm_trace=False)
'''
# Assemble a classifier
SVC = svm.SVC(kernel='rbf', gamma=3, C=2)
lda = LinearDiscriminantAnalysis()

mne.set_log_level('WARNING')
csp = CSP(n_components = 12, log=True, norm_trace=False)
#('auto', 'empirical', 'diagonal_fixed', 'ledoit_wolf', 'oas', 'shrunk', 'pca', 'factor_analysis', 'shrinkage')"
# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('CSP', csp), ('SVC', SVC)])
scores = cross_val_score(clf, epochs_data, labels, cv=cv, n_jobs=1)

# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                         class_balance))
'''

## Plot CSP patterns on full data for visualization

In [ ]:
# plot CSP patterns estimated on full data for visualization
csp.fit_transform(epochs_data, labels)

layout = read_layout('EEG1005')
csp.plot_patterns(epochs.info, layout=layout, ch_type='eeg',
                  units='Patterns (AU)', scalings=1e-9)
csp.plot_filters(epochs.info, layout=layout, ch_type='eeg', units='Patterns (AU)', scalings=1e-9)

## Get the desired spatial pattern and plot spectral density

In [ ]:
# get data
X = epochs.get_data()
labels = (epochs.events[:, -1] -1)
y = np.array(labels)

# compute spatial filtered spectrum
po = []
# Over all CSP filters
for x in X:
    po2 = []
    for filter in range(np.shape(csp.filters_)[1]):
        f,p = welch(np.dot(csp.filters_[filter,:].T,x), 500, nperseg=512)
        po2.append(p)
    po.append(po2)
po = np.array(po)
po = po.mean(axis=1)
''' For a single CSP Filter
for x in X:
    f,p = welch(np.dot(csp.filters_[0,:].T,x), 500, nperseg=512)
    po.append(p)
po = np.array(po)
'''
# prepare topoplot
_,epos,_,_,_ = _prepare_topo_plot(epochs,'eeg',None)
    
# plot first pattern
pattern = csp.patterns_[0,:]
pattern -= pattern.mean()
ix = np.argmax(abs(pattern))
# the pattern is sign invariant.
# invert it for display purpose
if pattern[ix]>0:
    sign = 1.0
else:
    sign = -1.0
    
fig, ax_topo = plt.subplots(1, 1, figsize=(22, 4))
#title = 'Spatial Pattern'
#fig.suptitle(title, fontsize=14)
img, _ = plot_topomap(sign*pattern,epos,show=False)
divider = make_axes_locatable(ax_topo)
# add axes for colorbar
ax_colorbar = divider.append_axes('right', size='10%', pad=0.5)
plt.colorbar(img, cax=ax_colorbar)

# plot spectrum
fix = (f>4) & (f<41)
ax_spectrum = divider.append_axes('right', size='300%', pad=3.2)
ax_spectrum.plot(f[fix],np.log(po[y==1][:,fix].mean(axis=0).T),'-r',lw=2)
ax_spectrum.plot(f[fix],np.log(po[y==0][:,fix].mean(axis=0).T),'-b',lw=2)
ax_spectrum.plot(f[fix],np.log(po[y==3][:,fix].mean(axis=0).T),'-y',lw=2)
ax_spectrum.plot(f[fix],np.log(po[y==2][:,fix].mean(axis=0).T),'-g',lw=2)
ax_spectrum.set_xlabel('Frequency (Hz)', fontsize='xx-large')
ax_spectrum.set_ylabel('Power (dB)', fontsize='xx-large')
plt.grid()
#plt.ylim(-6.5, -5.0)
plt.tick_params(labelsize=20)
plt.rc('xtick', labelsize=20)     
plt.rc('ytick', labelsize=20)
#plt.legend(['Left Attention','Right Attention'], fontsize='xx-large')
plt.legend(['Back Left Attention','Back Right Attention', 'Front Left Attention', 'Front Right Attention'], fontsize='xx-large')
#plt.title('Subject: Michael Balas')
plt.show()
#plt.savefig('spatial_pattern_subject_%02d.png' % subject ,bbox_inches='tight')

In [ ]:
# import a linear classifier from mne.decoding

X = epochs.pick_types(meg=False, eeg=True)
y = epochs.events[:, 2]

# Define a unique pipeline to sequentially:
clf = make_pipeline(
    Vectorizer(),                       # 1) vectorize across time and channels
    StandardScaler(),                   # 2) normalize features across trials
    LinearModel(
        LogisticRegression(solver='lbfgs')))  # 3) fits a logistic regression
clf.fit(X, y)

# Extract and plot patterns and filters
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, epochs.info, tmin=epochs.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s')

## Temporal Decoding

In [ ]:
X = epochs.get_data()  # MEG signals: n_epochs, n_meg_channels, n_times
y = epochs.events[:, 2]-1  # target: Audio left or right

clf = make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs'))

time_decod = SlidingEstimator(clf, n_jobs=1, verbose=True)
scores = cross_val_multiscore(time_decod, X, y, cv=3, n_jobs=1)

# Mean scores across cross-validation splits
scores = np.mean(scores, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(epochs.times, scores, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding')

In [ ]:
clf = make_pipeline(StandardScaler(),
                    LinearModel(LogisticRegression(solver='lbfgs')))
time_decod = SlidingEstimator(clf, n_jobs=1, scoring='accuracy', verbose=True)
time_decod.fit(X, y)

coef = get_coef(time_decod, 'patterns_', inverse_transform=True)
coef = np.mean(coef,axis=1)
# Show only the first four seconds
fourSecs = int((4.40565624999999983)*256)
evoked = mne.EvokedArray(coef[:,:fourSecs], epochs.info, tmin=epochs.times[0])
joint_kwargs = dict(ts_args=dict(time_unit='s'),
                    topomap_args=dict(time_unit='s'))
evoked.plot_joint(times=[-0.5,0,1,2,3,4], title='patterns',**joint_kwargs)

## Temporal Generalization

In [ ]:
# define the Temporal generalization object
clf = make_pipeline(StandardScaler(),
                    LinearModel(LogisticRegression(solver='lbfgs')))
time_gen = GeneralizingEstimator(clf, n_jobs=-1, scoring='accuracy',
                                 verbose=True)

scores = cross_val_multiscore(time_gen, epochs.get_data(), labels, cv=5, n_jobs=-1)

# Mean scores across cross-validation splits
scores = np.mean(scores, axis=0)

# Plot the diagonal (it's exactly the same as the time-by-time decoding above)
fig, ax = plt.subplots()
ax.plot(epochs.times, np.diag(scores), label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Decoding MEG sensors over time')

#### Plot the full generalization matrix

In [ ]:
fig, ax = plt.subplots(1, 1)
im = ax.imshow(scores, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=epochs.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('Temporal generalization')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
plt.colorbar(im, ax=ax)